In [ ]:
# 🛠️ Setup!pip install librosa torchaudio matplotlib einops -qfrom google.colab import drivedrive.mount('/content/drive')

In [ ]:
# 🗒️ ConfigDATA_DIR = '/content/drive/MyDrive/whale_denoising'NOISY_DIR = f'{DATA_DIR}/noisy'CLEAN_DIR = f'{DATA_DIR}/clean'

In [ ]:
# 📂 Clone your forked perch repo!git clone https://github.com/tgifford-usc/perch.git%cd perch!pip install -e .

In [ ]:
# 🧐 Load whale embedding modelimport syssys.path.append('chirp/projects/whale_demo')from agile_modeling_noaa_demo import load_modelembedding_model = load_model()

In [ ]:
# 🎵 Load audioimport globimport torchimport torchaudiodef load_audio(path):    wav, sr = torchaudio.load(path)    assert sr == 10000    return wav[0]def load_dataset(noisy_dir, clean_dir):    noisy_paths = sorted(glob.glob(noisy_dir + '/*.wav'))    clean_paths = sorted(glob.glob(clean_dir + '/*.wav'))    noisy = [load_audio(p) for p in noisy_paths]    clean = [load_audio(p) for p in clean_paths][:len(noisy)]    return noisy[:len(clean)], cleannoisy_data, clean_data = load_dataset(NOISY_DIR, CLEAN_DIR)

In [ ]:
# 📀 Datasetfrom torch.utils.data import Dataset, DataLoaderclass WhaleDenoiseDataset(Dataset):    def __init__(self, noisy, clean):        self.noisy = noisy        self.clean = clean    def __len__(self):        return len(self.noisy)    def __getitem__(self, idx):        return self.noisy[idx], self.clean[idx]train_dataset = WhaleDenoiseDataset(noisy_data, clean_data)train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
# 🧹 Denoiser modelimport torch.nn as nnclass Denoiser1D(nn.Module):    def __init__(self):        super().__init__()        self.net = nn.Sequential(            nn.Conv1d(1, 32, kernel_size=15, padding=7),            nn.ReLU(),            nn.Conv1d(32, 64, kernel_size=15, padding=7),            nn.ReLU(),            nn.Conv1d(64, 32, kernel_size=15, padding=7),            nn.ReLU(),            nn.Conv1d(32, 1, kernel_size=15, padding=7)        )    def forward(self, x):        return self.net(x)denoiser = Denoiser1D().cuda()optimizer = torch.optim.Adam(denoiser.parameters(), lr=1e-3)mse_loss = nn.MSELoss()

In [ ]:
# ↺ Training loopfrom tqdm import tqdmfor epoch in range(10):    denoiser.train()    total_loss = 0    for noisy, clean in tqdm(train_loader):        noisy = noisy.unsqueeze(1).cuda()        clean = clean.unsqueeze(1).cuda()        output = denoiser(noisy)        loss = mse_loss(output, clean)        with torch.no_grad():            embed_clean = embedding_model.get_embeddings(clean.squeeze(1)).detach()            embed_denoised = embedding_model.get_embeddings(output.squeeze(1))        embed_loss = ((embed_clean - embed_denoised)**2).mean()        total = loss + 0.1 * embed_loss        optimizer.zero_grad()        total.backward()        optimizer.step()        total_loss += total.item()    print(f'Epoch {epoch+1}, Loss = {total_loss/len(train_loader):.4f}')